In [178]:
'''
import spotipy
import spotipy.util as util
import os
import sys
import json
import webbrowser
from json.decoder import JSONDecodeError
'''

'\nimport spotipy\nimport spotipy.util as util\nimport os\nimport sys\nimport json\nimport webbrowser\nfrom json.decoder import JSONDecodeError\n'

In [179]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

In [180]:
# Authentication
cid = "97b1d5814fd2487da196df943dda834d"
secret = "87f620b5a6464e2d85d9fe1e32253d35"
client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [181]:
# EDIT THIS SECTION

#playlist_link = "https://open.spotify.com/playlist/6HGXewRDu4sH2qy8NYnNba?si=e41039b4ee764d3d"
#genres = "orchestral"
playlist_links = {
    "pop": "https://open.spotify.com/playlist/13Cm1hem9RE4v2ZOMJv34T?si=C9qRRpOQT4-tIqZlzERd6w&app_destination=copy-link",
    "hiphop": "https://open.spotify.com/playlist/5K9FlaF7V8Ib4X09rl23w6?si=XnauqM5BTiiqvp88B1bDwA&app_destination=copy-link",
    "rock": "https://open.spotify.com/playlist/5Wk9TcVaNE5yCyli90HmaR?si=7e1c58897b224157",
    "country": "https://open.spotify.com/playlist/6hjSKEoPqPLPTt3u0e9bLQ?si=0c844e194b8e467f",
    "electronic": "https://open.spotify.com/playlist/4ZMSlQbw13hExG4ztynNNV?si=1176e787add3496d",
    "R&B": "https://open.spotify.com/playlist/5WJXKWvPeWA9ubmvVsMTBh?si=YQ0Hnol7Q6S7sivtxHAssA&app_destination=copy-link",
    "metal": "https://open.spotify.com/playlist/2mDXGVXMG4ZMQaR26iZVC7?si=2beae20751ab4364",
    "jazz": "https://open.spotify.com/playlist/6GrLcuf2cf8g6h8lkZ0h7H?si=3b460ce380ff4ee6",
    "reggae": "https://open.spotify.com/playlist/6E5Fr9QU6yAety6Y6pw11u?si=28e0d5583c814bb0",
    "disco": "https://open.spotify.com/playlist/15X96mpCbP1ZiX8WIBqOhO?si=ba46abbe1146480f",
    "folk": "https://open.spotify.com/playlist/5fEekkUMaM2Le4FL38UuKx?si=b569e2ad8a264b6b",
    "orchestral":"https://open.spotify.com/playlist/6HGXewRDu4sH2qy8NYnNba?si=e41039b4ee764d3d"
}

#

#playlist_URI = playlist_link.split("/")[-1].split("?")[0]
#track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [182]:
def get_playlist_tracks(playlist_URI):
    results = sp.playlist_tracks(playlist_URI)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [183]:
trackdata = []
#trackids = set()
for genre, playlist_link in playlist_links.items():
    i = 0
    playlist_URI = playlist_link.split("/")[-1].split("?")[0]

    for track in get_playlist_tracks(playlist_URI):
    
        #URI
        track_uri = track["track"]["uri"]
        
        #Track name
        track_name = track["track"]["name"]

        #Popularity of the track
        track_pop = track["track"]["popularity"]

        #Explicitness of the track
        track_exp = track["track"]["explicit"]
        
        #Main Artist
        artist_uri = track["track"]["artists"][0]["uri"]
        #artist_info = sp.artist(artist_uri)
        
        #Name, popularity, genre
        artist_name = track["track"]["artists"][0]["name"]
        #artist_pop = artist_info["popularity"]
        #artist_genres = artist_info["genres"]
        
        #Album
        album = track["track"]["album"]["name"]
        
        features = {}
        audiofeatures = sp.audio_features(track_uri)[0]
        features["name"] = track_name
        features["artist"] = artist_name
        features["album"] = album
        features["popularity"] = track_pop
        features.update(audiofeatures)
        features["genre"]= genre
        trackdata.append(features)
        i += 1
mydf = pd.DataFrame(trackdata)
mydf = mydf.drop(columns = ["uri", "track_href", "analysis_url", "type"])


In [ ]:
mydf.to_csv("trackdata.csv", index = False)

In [218]:
duplicate = mydf["id"].duplicated(keep = False)
test = mydf[duplicate].sort_values(by=["name"])
genre_conflict_same_track = [2017,1067,1258,478,1069,5107,406,444,350,1339,5080,5136,254,3310,
                    1263,4821,2309,460,1219,63,3404,4804,3307,156,246,1253,1421,4797,
                    196,2337,1485,54,1145,2174,468,5094,3430,277,2203,2104,5212,1116,
                    5101,1234,2211,2191,497,5103,694,4052,167,1220,423,3326,400,1212,
                    1369,5173,5083,5097,2293,473,1010,1062,3221,2588,38]
print(len(genre_conflict_same_track))
tracksdf = mydf.drop(genre_conflict_same_track)

67


In [237]:
sorted = tracksdf.sort_values(["name","artist","popularity"], axis = 0)
genrecheck = []
rem = []
for i in range(len(sorted.index[:-1])):
    if sorted.iloc[i,16] == sorted.iloc[i+1,16]:
            if sorted.iloc[i,18] != sorted.iloc[i+1,18]:
                genrecheck.append((sorted.index[i+1],sorted.iloc[i,0], sorted.iloc[i,1], sorted.iloc[i,18],sorted.iloc[i+1,18]))
            rem.append(sorted.index[i])
sorted = sorted.drop(rem)

In [232]:
genrecheck

[(5116, 'Bad - 2012 Remaster', 'Michael Jackson', 'pop', 'disco'),
 (2750, 'Bitch Better Have My Money', 'Rihanna', 'hiphop', 'R&B'),
 (2210, 'By Your Side', 'Jonas Blue', 'pop', 'electronic'),
 (1565, 'Colder Heavens', 'Blanco White', 'folk', 'country'),
 (2168, 'Die Young', 'Kesha', 'pop', 'electronic'),
 (1276, 'Girls, Girls, Girls', 'Mötley Crüe', 'metal', 'rock'),
 (2270, 'I Feel It Coming', 'The Weeknd', 'pop', 'electronic'),
 (2198, 'Just Dance', 'Lady Gaga', 'pop', 'electronic'),
 (59, 'Lean On', 'Major Lazer', 'electronic', 'pop'),
 (2176, 'Memories (feat. Kid Cudi)', 'David Guetta', 'pop', 'electronic'),
 (2194, 'Middle', 'DJ Snake', 'pop', 'electronic'),
 (2721, 'Needed Me', 'Rihanna', 'hiphop', 'R&B'),
 (2726, 'One Dance', 'Drake', 'pop', 'R&B'),
 (1280, 'Rock You Like A Hurricane', 'Scorpions', 'metal', 'rock'),
 (51, 'See You Again (feat. Charlie Puth)', 'Wiz Khalifa', 'hiphop', 'pop'),
 (5108, 'Smooth Criminal - 2012 Remaster', 'Michael Jackson', 'pop', 'disco'),
 (1023,

In [238]:
# bad = disco 2
# bitch better have my money = hiphop 1
# by your side = pop 1
# colder heavens = folk 1
# die young = pop 1
# girls girls girls = rock 2
# I feel it coming = pop 1
# just dance = electronic 2
# lean on = electronic 1
# memories = electronic 2
# middle = pop 1
# needed me = R&B 2
# one dance = R&B 2
# rock you like a hurricane = rock 2
# see you again = pop 2
# smooth criminal = pop 1
# the only exception = pop 1
replacement_genre = [2,1,1,1,1,2,1,2,1,2,1,2,2,2,2,1,1]
for i in range(len(replacement_genre)):
    mytup = genrecheck[i]
    sorted.loc[mytup[0],"genre"] = mytup[2+replacement_genre[i]]

In [245]:

final = sorted.reset_index(drop = True)
final.to_csv("dupremoved_trackdata.csv", index = False)